## IMPORTS

In [1]:
import pandas as pd
import numpy as np

## LOAD THE DATA FILES

In [2]:
basics_url = "https://datasets.imdbws.com/title.basics.tsv.gz"
akas_url = "https://datasets.imdbws.com/title.akas.tsv.gz"
ratings_url = "https://datasets.imdbws.com/title.ratings.tsv.gz"

## LOAD AND FILTER BASICS DATA

In [3]:
basics = pd.read_csv(basics_url, sep='\t', low_memory=False)
basics.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0000001,short,Carmencita,Carmencita,0,1894,\N,1,"Documentary,Short"
1,tt0000002,short,Le clown et ses chiens,Le clown et ses chiens,0,1892,\N,5,"Animation,Short"
2,tt0000003,short,Pauvre Pierrot,Pauvre Pierrot,0,1892,\N,4,"Animation,Comedy,Romance"
3,tt0000004,short,Un bon bock,Un bon bock,0,1892,\N,12,"Animation,Short"
4,tt0000005,short,Blacksmith Scene,Blacksmith Scene,0,1893,\N,1,"Comedy,Short"


In [4]:
##Make a coppy of basics
basics1 = basics.copy()

In [5]:
## Replace Missing Valuess with NaN
basics1.replace({'\\N':np.nan}, inplace = True)

In [6]:
## Checcking to see if they have changed
basics1.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0000001,short,Carmencita,Carmencita,0,1894,NaN,1,"Documentary,Short"
1,tt0000002,short,Le clown et ses chiens,Le clown et ses chiens,0,1892,NaN,5,"Animation,Short"
2,tt0000003,short,Pauvre Pierrot,Pauvre Pierrot,0,1892,NaN,4,"Animation,Comedy,Romance"
3,tt0000004,short,Un bon bock,Un bon bock,0,1892,NaN,12,"Animation,Short"
4,tt0000005,short,Blacksmith Scene,Blacksmith Scene,0,1893,NaN,1,"Comedy,Short"


In [7]:
## Check for duplicates
basics1.duplicated().sum()

0

In [8]:
## Check missing values
basics1.isna().sum()

tconst                  0
titleType               0
primaryTitle           11
originalTitle          11
isAdult                 1
startYear         1319110
endYear           9651666
runtimeMinutes    6882999
genres             439371
dtype: int64

### Dropping rows with null values for runtime

In [9]:
basics1.dropna(subset = ['runtimeMinutes'],inplace=True)
basics1.isna().sum()

tconst                  0
titleType               0
primaryTitle            1
originalTitle           1
isAdult                 1
startYear          169432
endYear           2822811
runtimeMinutes          0
genres              76446
dtype: int64

### Dropping rows with missing values for genres

In [10]:
basics1.dropna(subset = ['genres'],inplace=True)
basics1.isna().sum()

tconst                  0
titleType               0
primaryTitle            1
originalTitle           1
isAdult                 0
startYear          164495
endYear           2747952
runtimeMinutes          0
genres                  0
dtype: int64

### Picking only Movies

In [11]:
basics1 = basics1.loc[basics1['titleType']== 'movie']

In [12]:
basics1['titleType'].value_counts()

movie    380377
Name: titleType, dtype: int64

### Specifying the years of the movies

In [13]:
basics1 = basics1.loc[basics1['startYear']>= '2000'] 
basics1 = basics1.loc[basics1['startYear']<= '2022']
basics1['startYear'].value_counts()

2017    14366
2018    14312
2019    14037
2016    13938
2015    13460
2014    13090
2022    12621
2013    12369
2021    12298
2012    11616
2020    11549
2011    10765
2010    10200
2009     9340
2008     8142
2007     6954
2006     6504
2005     5817
2004     5192
2003     4582
2002     4127
2001     3856
2000     3635
Name: startYear, dtype: int64

### Include only fictional movies (not from documentary genre)

In [14]:
is_documentary = basics1['genres'].str.contains('Documentary',case=False)
basics1 = basics1[~is_documentary]

## Inspect the AKAS Data

In [15]:
akas = pd.read_csv(akas_url, sep = '\t', low_memory = False)
akas.head()

,titleId,ordering,title,region,language,types,attributes,isOriginalTitle
0,tt0000001,1,Карменсіта,UA,\N,imdbDisplay,\N,0
1,tt0000001,2,Carmencita,DE,\N,\N,literal title,0
2,tt0000001,3,Carmencita - spanyol tánc,HU,\N,imdbDisplay,\N,0
3,tt0000001,4,Καρμενσίτα,GR,\N,imdbDisplay,\N,0
4,tt0000001,5,Карменсита,RU,\N,imdbDisplay,\N,0


In [16]:
## Make a copy of data
akas1 = akas.copy()

In [17]:
akas1 = akas1.loc[akas1['region']== 'US']

In [18]:
akas1['region'].value_counts()

US    1428057
Name: region, dtype: int64

### Replace "\N" with np.nan

In [19]:
akas1.replace({'\\N':np.nan}, inplace = True)

In [20]:
## Check to see if it worked
akas1.head()

,titleId,ordering,title,region,language,types,attributes,isOriginalTitle
5,tt0000001,6,Carmencita,US,NaN,imdbDisplay,NaN,0
14,tt0000002,7,The Clown and His Dogs,US,NaN,NaN,literal English title,0
33,tt0000005,10,Blacksmith Scene,US,NaN,imdbDisplay,NaN,0
36,tt0000005,1,Blacksmithing Scene,US,NaN,alternative,NaN,0
41,tt0000005,6,Blacksmith Scene #1,US,NaN,alternative,NaN,0


In [21]:
keepers =basics1['tconst'].isin(akas['titleId'])
keepers

34803      True
61116      True
67669      True
77964      True
86801      True
           ... 
9756307    True
9756316    True
9756355    True
9756400    True
9756484    True
Name: tconst, Length: 147191, dtype: bool

In [22]:
basics1 = basics1[keepers]
basics1

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
34803,tt0035423,movie,Kate & Leopold,Kate & Leopold,0,2001,NaN,118,"Comedy,Fantasy,Romance"
61116,tt0062336,movie,The Tango of the Widower and Its Distorting Mi...,El tango del viudo y su espejo deformante,0,2020,NaN,70,Drama
67669,tt0069049,movie,The Other Side of the Wind,The Other Side of the Wind,0,2018,NaN,122,Drama
77964,tt0079644,movie,November 1828,November 1828,0,2001,NaN,140,"Drama,War"
86801,tt0088751,movie,The Naked Monster,The Naked Monster,0,2005,NaN,100,"Comedy,Horror,Sci-Fi"
...,...,...,...,...,...,...,...,...,...
9756307,tt9916170,movie,The Rehearsal,O Ensaio,0,2019,NaN,51,Drama
9756316,tt9916190,movie,Safeguard,Safeguard,0,2020,NaN,95,"Action,Adventure,Thriller"
9756355,tt9916270,movie,Il talento del calabrone,Il talento del calabrone,0,2020,NaN,84,Thriller
9756400,tt9916362,movie,Coven,Akelarre,0,2020,NaN,92,"Drama,History"


In [23]:
ratings = pd.read_csv(ratings_url, sep = '\t', low_memory = False)
ratings.head()

,tconst,averageRating,numVotes
0,tt0000001,5.7,1965
1,tt0000002,5.8,262
2,tt0000003,6.5,1805
3,tt0000004,5.6,178
4,tt0000005,6.2,2603


In [24]:
ratings1 = ratings.copy()

In [25]:
keepers =ratings1['tconst'].isin(akas['titleId'])
keepers

0           True
1           True
2           True
3           True
4           True
           ...  
1298299     True
1298300     True
1298301    False
1298302    False
1298303    False
Name: tconst, Length: 1298304, dtype: bool

In [26]:
ratings1 = ratings1[keepers]
ratings1 

,tconst,averageRating,numVotes
0,tt0000001,5.7,1965
1,tt0000002,5.8,262
2,tt0000003,6.5,1805
3,tt0000004,5.6,178
4,tt0000005,6.2,2603
...,...,...,...
1298283,tt9916460,9.4,18
1298286,tt9916538,8.6,7
1298287,tt9916544,6.9,62
1298299,tt9916730,8.3,10


In [27]:
# example making new folder with os
import os
os.makedirs('Data/',exist_ok=True) 
# Confirm folder created
os.listdir("Data/")

['tmdb_api_results_2000.json',
 'final_tmdb_data_2000.csv.gz',
 'tmdb_api_results_2001.json',
 'final_tmdb_data_2001.csv.gz',
 '.ipynb_checkpoints',
 'tmdb_results_combined.csv.gz']

In [28]:
basics1.to_csv("Data/title_basics1.csv.gz",compression='gzip',index=False)
basics1.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
34803,tt0035423,movie,Kate & Leopold,Kate & Leopold,0,2001,NaN,118,"Comedy,Fantasy,Romance"
61116,tt0062336,movie,The Tango of the Widower and Its Distorting Mi...,El tango del viudo y su espejo deformante,0,2020,NaN,70,Drama
67669,tt0069049,movie,The Other Side of the Wind,The Other Side of the Wind,0,2018,NaN,122,Drama
77964,tt0079644,movie,November 1828,November 1828,0,2001,NaN,140,"Drama,War"
86801,tt0088751,movie,The Naked Monster,The Naked Monster,0,2005,NaN,100,"Comedy,Horror,Sci-Fi"


In [29]:
akas1.to_csv("Data/title_akas1.csv.gz", compression = 'gzip', index = False)
akas1.head()

,titleId,ordering,title,region,language,types,attributes,isOriginalTitle
5,tt0000001,6,Carmencita,US,NaN,imdbDisplay,NaN,0
14,tt0000002,7,The Clown and His Dogs,US,NaN,NaN,literal English title,0
33,tt0000005,10,Blacksmith Scene,US,NaN,imdbDisplay,NaN,0
36,tt0000005,1,Blacksmithing Scene,US,NaN,alternative,NaN,0
41,tt0000005,6,Blacksmith Scene #1,US,NaN,alternative,NaN,0


In [30]:
ratings1.to_csv("Data/title_ratings1.csv.gz", compression = 'gzip', index = False)
ratings1.head()

,tconst,averageRating,numVotes
0,tt0000001,5.7,1965
1,tt0000002,5.8,262
2,tt0000003,6.5,1805
3,tt0000004,5.6,178
4,tt0000005,6.2,2603
